In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/sw_contest"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/sw_contest


In [2]:
!pip install torchmetrics --quiet

In [3]:
!pip install segmentation_models_pytorch --quiet

In [4]:
!pip install lightning --quiet

In [5]:
import os

import cv2
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchmetrics import Dice
import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2

import lightning as L
from lightning.pytorch.utilities.types import TRAIN_DATALOADERS, EVAL_DATALOADERS
from lightning.pytorch.loggers import WandbLogger

## Decode mask

In [6]:
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [7]:
def make_binary_mask_images(img_name, mask_rle, save_path):
    img = cv2.imread(os.path.join("./train_img", img_name))

    img = rle_decode(mask_rle, (img.shape[0], img.shape[1]))
    cv2.imwrite(os.path.join(save_path, img_name), img)

    return img

In [8]:
df = pd.read_csv("./train.csv")

mask_path = "./train_mask"
os.makedirs(mask_path, exist_ok=True)

for idx, row in df.iterrows():
   img_name = row["img_path"].split('/')[-1]
   mask_rle = row["mask_rle"]
   make_binary_mask_images(img_name, mask_rle, mask_path)

KeyboardInterrupt: ignored

## Validation offline augmentation
crop with (256 * 256)

In [ ]:
train_df = pd.read_csv("./train.csv")
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=0)

In [ ]:
print(len(train_df))
print(len(val_df))

5712
1428


In [ ]:
def crop_img(img, img_size=256):
    img_list = []

    y_cnt = 0
    while True:
        start_y = y_cnt * img_size
        end_y = (y_cnt + 1) * img_size

        if end_y > 1024:
            break

        x_cnt = 0
        while True:
            start_x = x_cnt * img_size
            end_x = (x_cnt + 1) * img_size

            if end_x > 1024:
                break

            temp_img = img[start_x:end_x, start_y:end_y, :]
            x_cnt += 1
            img_list.append(temp_img)

        y_cnt += 1

    return img_list

In [ ]:
new_data_list = []
for idx, row in val_df.iterrows():
    img_name = row["img_path"].split("/")[-1]
    img_path = os.path.join("./train_img", img_name)
    mask_path = os.path.join("./train_mask", img_name)
    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path)

    images = crop_img(img)
    masks = crop_img(mask)

    for idx, (img, mask) in enumerate(zip(images, masks)):
        new_img_name = img_name[:-4] + "_" + str(idx).zfill(2) + ".png"
        new_data_list.append({"img_id": new_img_name[:-4]})

        cv2.imwrite(os.path.join("./train_img3", new_img_name), img)
        cv2.imwrite(os.path.join("./train_mask3", new_img_name), mask)



In [ ]:
new_data_list

In [ ]:
new_data_path = []

for filename in os.listdir('./train_img3'):
  new_data_path.append({'img_path': "./train_img3/" + filename})

print(new_data_path[0])

{'img_path': './train_img3/TRAIN_4972_00.png'}


In [ ]:
new_val_df = pd.DataFrame(new_data_list)


In [ ]:
new_val_df2 = pd.DataFrame(new_data_path)


In [ ]:
val = pd.concat([new_val_df, new_val_df2], axis=1)
print(val)

              img_id                        img_path
0      TRAIN_4972_00  ./train_img3/TRAIN_4972_00.png
1      TRAIN_4972_01  ./train_img3/TRAIN_4972_01.png
2      TRAIN_4972_02  ./train_img3/TRAIN_4972_02.png
3      TRAIN_4972_03  ./train_img3/TRAIN_4972_03.png
4      TRAIN_4972_04  ./train_img3/TRAIN_4972_04.png
...              ...                             ...
22843  TRAIN_0346_11  ./train_img3/TRAIN_0346_11.png
22844  TRAIN_0346_12  ./train_img3/TRAIN_0346_12.png
22845  TRAIN_0346_13  ./train_img3/TRAIN_0346_13.png
22846  TRAIN_0346_14  ./train_img3/TRAIN_0346_14.png
22847  TRAIN_0346_15  ./train_img3/TRAIN_0346_15.png

[22848 rows x 2 columns]


In [ ]:
val.to_csv("./val.csv", index=False)

## Train Model

In [7]:
configs = {
    "model": {
        "encoder_name": "timm-regnety_320",
        "encoder_weights": "imagenet",
        "in_channels": 3,
        "classes": 1
    },
    "data": {
        "root": ".",
        "batch_size": 32
    },
}

#### Dataset

In [8]:
class DefaultTransforms:
    def __init__(self) -> None:
        super().__init__()

    def train_transform(self):
        return A.Compose(
            [
                A.OneOf(
                    [
                        A.RandomBrightness(p=1),
                        A.RandomBrightnessContrast(p=1),
                        A.Emboss(p=1),
                        A.RandomShadow(p=1),
                        A.NoOp(),
                    ],
                    p=1,
                ),
                A.OneOf(
                    [
                        A.Blur(p=1),
                        A.AdvancedBlur(p=1),
                        A.MotionBlur(p=1),
                    ],
                    p=0.6,
                ),
                A.OneOf(
                    [
                        A.NoOp(),
                        A.HorizontalFlip(p=0.5),
                        A.VerticalFlip(p=0.5),
                        A.ShiftScaleRotate(p=0.5),
                        A.Rotate(limit=90, p=1, border_mode=cv2.BORDER_REPLICATE),
                        A.RandomRotate90(p=1)
                    ],
                    p=1,
                ),
                A.RandomCrop(224, 224),
                A.Normalize(),
                ToTensorV2(transpose_mask=True)
            ]
        )

    def val_transform(self):
        return A.Compose(
            [
                A.Resize(224, 224),
                A.Normalize(),
                ToTensorV2(transpose_mask=True)
            ]
        )

    def test_transform(self):
        return A.Compose(
            [
                A.Normalize(),
                ToTensorV2(transpose_mask=True)
            ]
        )

In [9]:
class SatelliteDataset(Dataset):
    def __init__(self, root, df, transform=None, train=False, val=False, infer=False):
        self.root = root
        self.data = df
        self.train = train
        self.val = val
        self.infer = infer
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]

        if self.train:
            image = cv2.imread(os.path.join("./train_img", img_name + ".png"))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            mask = cv2.imread(os.path.join("./train_mask", img_name + ".png"), cv2.IMREAD_GRAYSCALE)

            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']


            return image, mask

        elif self.val:
            image = cv2.imread(os.path.join("./train_img3", img_name + ".png"))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            mask = cv2.imread(os.path.join("./train_mask3", img_name + ".png"), cv2.IMREAD_GRAYSCALE)

            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']

            return image, mask

        elif self.infer:
            image = cv2.imread(os.path.join("./test_img", img_name + ".png"))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            if self.transform:
                image = self.transform(image=image)['image']
            return image


class SatelliteDataModule(L.LightningDataModule):
    def __init__(
        self,
        root: str,
        batch_size: int,
    ) -> None:
        super().__init__()
        self.root = root
        self.transforms = DefaultTransforms()
        self.train_transform = self.transforms.train_transform()
        self.val_transform = self.transforms.val_transform()
        self.test_transform = self.transforms.test_transform()
        self.batch_size = batch_size

    def setup(self, stage: str) -> None:
        train_df = pd.read_csv(os.path.join(self.root, "train.csv"))
        test_df = pd.read_csv(os.path.join(self.root, "test.csv"))
        val_df = pd.read_csv(os.path.join(self.root, "val.csv"))

        self.train_dataset = SatelliteDataset(
            self.root, df=train_df, train=True, transform=self.train_transform
        )
        self.val_dataset = SatelliteDataset(
            self.root, df=val_df, val=True, transform=self.val_transform
        )
        self.test_dataset = SatelliteDataset(
            self.root, df=test_df, infer=False, transform=self.test_transform
        )

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=8
        )

    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(
            self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=8
        )

    def test_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(
            self.test_dataset, batch_size=32, shuffle=False, num_workers=8
        )


#### Model

In [10]:
class SMP(nn.Module):
    def __init__(self, encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=1):
        super().__init__()

        self.model = smp.UnetPlusPlus(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=classes,
        )

    def forward(self, x):
        x = self.model(x)
        return x

#### Training

In [11]:
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        inputs = F.sigmoid(inputs) # sigmoid를 통과한 출력이면 주석처리

        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

        return 1 - dice


In [12]:
class LitSeg(L.LightningModule):
    def __init__(
        self,
    ) -> None:
        super().__init__()
        self.save_hyperparameters(ignore=["net", "loss_module", "metric_module"])

        self.net = SMP(
            encoder_name=configs["model"]["encoder_name"],
            encoder_weights=configs["model"]["encoder_weights"],
            in_channels=configs["model"]["in_channels"],
            classes=configs["model"]["classes"],
        )

        self.loss_module = DiceLoss()
        self.metric_module = Dice()

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=1)

        return [optimizer], [scheduler]

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        img, mask = batch
        pred = self(img)

        loss = self.loss_module(pred, mask)
        self.log("train/loss", loss.item())

        return loss

    def validation_step(self, batch, batch_idx):
        img, mask = batch
        pred = self(img).squeeze()

        loss = self.loss_module(pred, mask)
        self.log("val/loss", loss.item(), on_epoch=True, on_step=False)

        dice_score = self.metric_module(pred, mask)
        self.log("val/dice_score", dice_score, on_epoch=True, on_step=False, prog_bar=True)

        self.logger.log_image(
            key="results",
            images=[img.unbind(dim=0)[0], pred.unbind(dim=0)[0], mask.float().unbind(dim=0)[0]],
            caption=["image", "pred", "mask"]
        )

    def on_test_start(self):
        self.result = []

    def test_step(self, batch, batch_idx):
        img = batch
        pred = self(img)

        pred = torch.sigmoid(pred).cpu().numpy()
        pred = np.squeeze(pred, axis=1)
        pred = (pred > 0.35).astype(np.uint8) # Threshold = 0.35

        for i in range(len(img)):
            mask_rle = self.rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                self.result.append(-1)
            else:
                self.result.append(mask_rle)

    def on_test_end(self):
        submit = pd.read_csv('./sample_submission.csv')
        submit['mask_rle'] = self.result
        submit.to_csv('./submit_0.8.csv', index=False)

    def rle_encode(self, mask):
        pixels = mask.flatten()
        pixels = np.concatenate([[0], pixels, [0]])
        runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
        runs[1::2] -= runs[::2]
        return ' '.join(str(x) for x in runs)

    #
    # def load_from_checkpoint(cls, checkpoint_path):
    #     model = cls()  # LitSeg 클래스의 인스턴스를 생성합니다.
    #     checkpoint = torch.load(checkpoint_path)
    #     model.load_state_dict(checkpoint['state_dict'])
    #     return model

In [ ]:
#34261ba713c68009e4fd6f3a41e097d1161e7adc

In [13]:
!pip install wandb --quiet


In [20]:
logger = WandbLogger(project="DACON")
model = LitSeg()
data_module = SatelliteDataModule(
    root=configs["data"]["root"],
    batch_size=configs["data"]["batch_size"]
)

trainer = L.Trainer(
    accelerator="gpu",
    precision="16-mixed",
    logger=logger,
    max_epochs=1,
    devices=1
)





INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# from lightning.pytorch.loggers import WandbLogger
# import lightning as L
# import wandb

# logger = WandbLogger(project="DACON", name='EarlyStoppingADam')
# model = LitSeg()
# data_module = SatelliteDataModule(
#     root=configs["data"]["root"],
#     batch_size=configs["data"]["batch_size"]
# )

# trainer = L.Trainer(
#     accelerator="gpu",
#     precision="16-mixed",
#     logger=logger,
#     max_epochs=1,
#     devices=1
# )

# trainer.fit(model=model, datamodule=data_module)

# trainer.save_checkpoint('EarlyStoppingADam-32-0.001.pth')
# wandb.save('EarlyStoppingADam-32-0.001.pth')

# wandb.restore('EarlyStoppingADam-32-0.001.pth')
# model.load_from_checkpoint('EarlyStoppingADam-32-0.001.pth')

In [21]:
from lightning.pytorch.loggers import WandbLogger
import lightning as L

logger = WandbLogger(project="DACON", name='EarlyStoppingADam-32-0.001')
model = LitSeg()
data_module = SatelliteDataModule(
    root=configs["data"]["root"],
    batch_size=configs["data"]["batch_size"]
)

trainer = L.Trainer(
    accelerator="gpu",
    precision="16-mixed",
    logger=logger,
    max_epochs=1,
    devices=1
)

trainer.fit(model=model, datamodule=data_module)



INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type     | Params
-------------------------------------------
0 | net           | SMP      | 179 M 
1 | loss_module   | DiceLoss | 0     
2 | metric_module | Dice     | 0     
-----------------------------

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


OSError: ignored

In [28]:
trainer.test(model, data_module)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Testing: 0it [00:00, ?it/s]

TypeError: ignored

In [27]:
loaded_model = LitSeg()
loaded_model.load_state_dict(torch.load('EarlyStoppingADam-32-0.001.pth'))
trainer.test(model=loaded_model, datamodule=data_module)


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Testing: 0it [00:00, ?it/s]

TypeError: ignored

In [25]:
import wandb
wandb.restore('EarlyStoppingADam-32-0.001.pth')
model.load_from_checkpoint('EarlyStoppingADam-32-0.001.pth')

#model = LitSeg.load_from_checkpoint("DACON/dashing-eon-11")
trainer.test(model=model, datamodule=data_module)

ValueError: ignored

![picture](https://github.com/silverstar0727/artifacts/blob/main/dacon-buliding-wandb.png?raw=true)

In [ ]:
model = LitSeg.load_from_checkpoint()
trainer.test(model=model, datamodule=data_module)